# Question1

from bs4 import BeautifulSoup
import requests
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [41]:
url = "https://en.wikipedia.org/w/index.php?title=List_of_postal_codes_of_Canada:_M&direction=prev&oldid=926287641"
source = requests.get(url).text

In [42]:
soup = BeautifulSoup(source, "xml")

In [43]:
table = soup.find("table")

In [44]:
df = pd.DataFrame(columns = ['PostalCode', 'Borough', 'Neighborhood'])

In [45]:
for tr_cell in table.find_all('tr'):
    row_data = []
    for td_cell in tr_cell.find_all('td'):
        row_data.append(td_cell.text.strip())
    if len(row_data) == 3:
        df.loc[len(df)] = row_data

In [46]:
df.head()

PostalCode           Borough      Neighborhood
0        M1A      Not assigned      Not assigned
1        M2A      Not assigned      Not assigned
2        M3A        North York         Parkwoods
3        M4A        North York  Victoria Village
4        M5A  Downtown Toronto      Harbourfront

In [50]:
df=df[df['Borough']!='Not assigned']
df[['Neighborhood']=='Not assigned']=df['Borough']
df.head(5)

PostalCode           Borough      Neighborhood             False
2        M3A        North York         Parkwoods        North York
3        M4A        North York  Victoria Village        North York
4        M5A  Downtown Toronto      Harbourfront  Downtown Toronto
5        M5A  Downtown Toronto       Regent Park  Downtown Toronto
6        M6A        North York  Lawrence Heights        North York

In [51]:
temp_df=df.groupby('PostalCode')['Neighborhood'].apply(lambda x: "%s" % ', '.join(x))
temp_df=temp_df.reset_index(drop=False)
temp_df.rename(columns={'Neighborhood':'Neighborhood_joined'},inplace=True)

In [52]:
df_merge = pd.merge(df, temp_df, on='PostalCode')
df_merge.drop(['Neighborhood'],axis=1,inplace=True)
df_merge.drop_duplicates(inplace=True)
df_merge.rename(columns={'Neighborhood_joined':'Neighborhood'},inplace=True)
df_merge.head()

PostalCode           Borough             False  \
0        M3A        North York        North York   
1        M4A        North York        North York   
2        M5A  Downtown Toronto  Downtown Toronto   
4        M6A        North York        North York   
6        M7A      Queen's Park      Queen's Park   

                       Neighborhood  
0                         Parkwoods  
1                  Victoria Village  
2         Harbourfront, Regent Park  
4  Lawrence Heights, Lawrence Manor  
6                      Not assigned

In [53]:
df_merge.shape

(103, 4)

# Question 2

!pip install geocoder
import geocoder # import geocoder


def get_geocode(postal_code):
    # initialize your variable to None
    lat_lng_coords = None
    while(lat_lng_coords is None):
        g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
        lat_lng_coords = g.latlng
    latitude = lat_lng_coords[0]
    longitude = lat_lng_coords[1]
    return latitude,longitude

In [55]:
geo_df=pd.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/labs_v1/Geospatial_Coordinates.csv")

geo_df.head()

geo_df.rename(columns={'Postal Code':'PostalCode'},inplace=True)
geo_merged = pd.merge(geo_df, df_merge, on='PostalCode')
geo_data=geo_merged[['PostalCode','Borough','Neighborhood','Latitude','Longitude']]
geo_data.head()

PostalCode      Borough                            Neighborhood   Latitude  \
0        M1B  Scarborough                          Rouge, Malvern  43.806686   
1        M1C  Scarborough  Highland Creek, Rouge Hill, Port Union  43.784535   
2        M1E  Scarborough       Guildwood, Morningside, West Hill  43.763573   
3        M1G  Scarborough                                  Woburn  43.770992   
4        M1H  Scarborough                               Cedarbrae  43.773136   

   Longitude  
0 -79.194353  
1 -79.160497  
2 -79.188711  
3 -79.216917  
4 -79.239476

# Question 3

import numpy as np
import os
from sklearn.cluster import KMeans
import folium 
from geopy.geocoders import Nominatim 
import matplotlib.cm as cm
import matplotlib.colors as colors

In [57]:
toronto_data=geo_data[geo_data['Borough'].str.contains("Toronto")]
toronto_data.head()

PostalCode          Borough                    Neighborhood   Latitude  \
37        M4E     East Toronto                     The Beaches  43.676357   
41        M4K     East Toronto    The Danforth West, Riverdale  43.679557   
42        M4L     East Toronto  The Beaches West, India Bazaar  43.668999   
43        M4M     East Toronto                 Studio District  43.659526   
44        M4N  Central Toronto                   Lawrence Park  43.728020   

    Longitude  
37 -79.293031  
41 -79.352188  
42 -79.315572  
43 -79.340923  
44 -79.388790

In [58]:
CLIENT_ID = 'DYOJFWB2RDCT2XVXBQLND5FIOD1S1VJ2QBKXT2ADJ5UBFCCN' # your Foursquare ID
CLIENT_SECRET = 'WYK0FEPFXEEWK0MLGFL2T1EYPQZ3PPR0FSUQJAQI0SFLRRPJ' # your Foursquare Secret
VERSION = '20180605'

In [59]:
LIMIT = 100
radius = 500

def getNearbyVenues(names, latitudes, longitudes):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [60]:
toronto_venues = getNearbyVenues(names=toronto_data['Neighborhood'],
                                   latitudes=toronto_data['Latitude'],
                                   longitudes=toronto_data['Longitude']
                                  )

The Beaches
The Danforth West, Riverdale
The Beaches West, India Bazaar
Studio District
Lawrence Park
Davisville North
North Toronto West
Davisville
Moore Park, Summerhill East
Deer Park, Forest Hill SE, Rathnelly, South Hill, Summerhill West
Rosedale
Cabbagetown, St. James Town
Church and Wellesley
Harbourfront, Regent Park
Ryerson, Garden District
St. James Town
Berczy Park
Central Bay Street
Adelaide, King, Richmond
Harbourfront East, Toronto Islands, Union Station
Design Exchange, Toronto Dominion Centre
Commerce Court, Victoria Hotel
Roselawn
Forest Hill North, Forest Hill West
The Annex, North Midtown, Yorkville
Harbord, University of Toronto
Chinatown, Grange Park, Kensington Market
CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara
Stn A PO Boxes 25 The Esplanade
First Canadian Place, Underground city
Christie
Dovercourt Village, Dufferin
Little Portugal, Trinity
Brockton, Exhibition Place, Parkdale Village
High Park, The 

In [61]:
toronto_venues.head()

Neighborhood  Neighborhood Latitude  Neighborhood Longitude  \
0  The Beaches              43.676357              -79.293031   
1  The Beaches              43.676357              -79.293031   
2  The Beaches              43.676357              -79.293031   
3  The Beaches              43.676357              -79.293031   
4  The Beaches              43.676357              -79.293031   

                                Venue  Venue Latitude  Venue Longitude  \
0                   Glen Manor Ravine       43.676821       -79.293942   
1  The Big Carrot Natural Food Market       43.678879       -79.297734   
2                 Grover Pub and Grub       43.679181       -79.297215   
3                       Upper Beaches       43.680563       -79.292869   
4                          Dip 'n Sip       43.678897       -79.297745   

      Venue Category  
0              Trail  
1  Health Food Store  
2                Pub  
3       Neighborhood  
4        Coffee Shop

In [62]:
toronto_venues.groupby('Neighborhood').count()

Neighborhood Latitude  \
Neighborhood                                                                
Adelaide, King, Richmond                                               94   
Berczy Park                                                            59   
Brockton, Exhibition Place, Parkdale Village                           22   
Business Reply Mail Processing Centre 969 Eastern                      15   
CN Tower, Bathurst Quay, Island airport, Harbou...                     17   
Cabbagetown, St. James Town                                            45   
Central Bay Street                                                     64   
Chinatown, Grange Park, Kensington Market                              63   
Christie                                                               15   
Church and Wellesley                                                   78   
Commerce Court, Victoria Hotel                                        100   
Davisville                                                             37   
Davisville North                                                        8   
Deer Park, Forest Hill SE, Rathnelly, South Hil...                     15   
Design Exchange, Toronto Dominion Centre                              100   
Dovercourt Village, Dufferin                                           15   
First Canadian Place, Underground city                                100   
Forest Hill North, Forest Hill West                                     4   
Harbord, University of Toronto                                         32   
Harbourfront East, Toronto Islands, Union Station                     100   
Harbourfront, Regent Park                                              45   
High Park, The Junction South                                          24   
Lawrence Park                                                           4   
Little Portugal, Trinity                                               43   
Moore Park, Summerhill East                                             1   
North Toronto West                                                     19   
Parkdale, Roncesvalles                                                 15   
Rosedale                                                                4   
Roselawn                                                                2   
Runnymede, Swansea                                                     33   
Ryerson, Garden District                                              100   
St. James Town                                                         82   
Stn A PO Boxes 25 The Esplanade                                       100   
Studio District                                                        36   
The Annex, North Midtown, Yorkville                                    22   
The Beaches                                                             5   
The Beaches West, India Bazaar                                         19   
The Danforth West, Riverdale                                           42   

                                                    Neighborhood Longitude  \
Neighborhood                                                                 
Adelaide, King, Richmond                                                94   
Berczy Park                                                             59   
Brockton, Exhibition Place, Parkdale Village                            22   
Business Reply Mail Processing Centre 969 Eastern                       15   
CN Tower, Bathurst Quay, Island airport, Harbou...                      17   
Cabbagetown, St. James Town                                             45   
Central Bay Street                                                      64   
Chinatown, Grange Park, Kensington Market                               63   
Christie                                                                15   
Church and Wellesley                                                    78   
Commerce Court, Victoria Hotel     

In [63]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")
toronto_onehot.drop(['Neighborhood'],axis=1,inplace=True) 
toronto_onehot.insert(loc=0, column='Neighborhood', value=toronto_venues['Neighborhood'] )
toronto_onehot.shape

(1579, 231)

In [64]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped.head()

Neighborhood  Adult Boutique  \
0                           Adelaide, King, Richmond             0.0   
1                                        Berczy Park             0.0   
2       Brockton, Exhibition Place, Parkdale Village             0.0   
3  Business Reply Mail Processing Centre 969 Eastern             0.0   
4  CN Tower, Bathurst Quay, Island airport, Harbo...             0.0   

    Airport  Airport Food Court  Airport Gate  Airport Lounge  \
0  0.000000            0.000000      0.000000        0.000000   
1  0.000000            0.000000      0.000000        0.000000   
2  0.000000            0.000000      0.000000        0.000000   
3  0.000000            0.000000      0.000000        0.000000   
4  0.058824            0.058824      0.058824        0.117647   

   Airport Service  Airport Terminal  American Restaurant  Antique Shop  \
0         0.000000          0.000000             0.021277           0.0   
1         0.000000          0.000000             0.000000           0.0   
2         0.000000          0.000000             0.000000           0.0   
3         0.000000          0.000000             0.000000           0.0   
4         0.117647          0.117647             0.000000           0.0   

   Aquarium  Art Gallery  Art Museum  Arts & Crafts Store  Asian Restaurant  \
0       0.0     0.010638    0.010638                  0.0          0.010638   
1       0.0     0.016949    0.000000                  0.0          0.000000   
2       0.0     0.000000    0.000000                  0.0          0.000000   
3       0.0     0.000000    0.000000                  0.0          0.000000   
4       0.0     0.000000    0.000000                  0.0          0.000000   

   Athletics & Sports  Auto Workshop  BBQ Joint  Baby Store  Bagel Shop  \
0                 0.0       0.000000        0.0         0.0    0.000000   
1                 0.0       0.000000        0.0         0.0    0.016949   
2                 0.0       0.000000        0.0         0.0    0.000000   
3                 0.0       0.066667        0.0         0.0    0.000000   
4                 0.0       0.000000        0.0         0.0    0.000000   

     Bakery      Bank       Bar  Baseball Stadium  Basketball Stadium  \
0  0.010638  0.010638  0.010638               0.0            0.000000   
1  0.050847  0.000000  0.000000               0.0            0.016949   
2  0.045455  0.000000  0.045455               0.0            0.000000   
3  0.000000  0.000000  0.000000               0.0            0.000000   
4  0.000000  0.000000  0.058824               0.0            0.000000   

      Beach  Bed & Breakfast  Beer Bar  Beer Store  Belgian Restaurant  \
0  0.000000              0.0  0.000000         0.0                 0.0   
1  0.016949              0.0  0.033898         0.0                 0.0   
2  0.000000              0.0  0.000000         0.0                 0.0   
3  0.000000              0.0  0.000000         0.0                 0.0   
4  0.000000              0.0  0.000000         0.0                 0.0   

     Bistro  Boat or Ferry  Bookstore  Boutique  Brazilian Restaurant  \
0  0.000000       0.000000   0.021277  0.000000              0.010638   
1  0.016949       0.000000   0.000000  0.000000              0.000000   
2  0.000000       0.000000   0.000000  0.000000              0.000000   
3  0.000000       0.000000   0.000000  0.000000              0.000000   
4  0.000000       0.058824   0.000000  0.058824              0.000000   

   Breakfast Spot   Brewery  Bubble Tea Shop  Burger Joint  Burrito Place  \
0        0.010638  0.000000              0.0      0.010638       0.021277   
1        0.016949  0.000000              0.0      0.000000       0.000000   
2        0.090909  0.000000              0.0      0.000000       0.045455   
3        0.000000  0.066667              0.0      0.000000       0.066667   
4        0.000000  0.000000              0.0      0.000000       0.000000   

   Bus Line  Business Service  Butcher      Café  Caj

In [65]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [66]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

Neighborhood 1st Most Common Venue  \
0                           Adelaide, King, Richmond           Coffee Shop   
1                                        Berczy Park           Coffee Shop   
2       Brockton, Exhibition Place, Parkdale Village                  Café   
3  Business Reply Mail Processing Centre 969 Eastern    Light Rail Station   
4  CN Tower, Bathurst Quay, Island airport, Harbo...        Airport Lounge   

  2nd Most Common Venue 3rd Most Common Venue 4th Most Common Venue  \
0                  Café            Restaurant       Thai Restaurant   
1          Cocktail Bar                Bakery            Restaurant   
2        Breakfast Spot           Coffee Shop                   Gym   
3            Comic Shop         Auto Workshop                  Park   
4       Airport Service      Airport Terminal              Boutique   

  5th Most Common Venue 6th Most Common Venue 7th Most Common Venue  \
0         Deli / Bodega        Clothing Store                   Gym   
1           Cheese Shop    Seafood Restaurant              Pharmacy   
2            Restaurant               Stadium             Nightclub   
3            Restaurant            Skate Park         Burrito Place   
4               Airport   Rental Car Location           Coffee Shop   

    8th Most Common Venue 9th Most Common Venue 10th Most Common Venue  
0     American Restaurant         Burrito Place                  Hotel  
1          Farmers Market              Beer Bar           Concert Hall  
2  Furniture / Home Store                   Bar                 Bakery  
3          Farmers Market  Fast Food Restaurant                Brewery  
4        Sculpture Garden         Boat or Ferry                    Bar

In [67]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [68]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = toronto_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head()

PostalCode          Borough                    Neighborhood   Latitude  \
37        M4E     East Toronto                     The Beaches  43.676357   
41        M4K     East Toronto    The Danforth West, Riverdale  43.679557   
42        M4L     East Toronto  The Beaches West, India Bazaar  43.668999   
43        M4M     East Toronto                 Studio District  43.659526   
44        M4N  Central Toronto                   Lawrence Park  43.728020   

    Longitude  Cluster Labels 1st Most Common Venue 2nd Most Common Venue  \
37 -79.293031               0                 Trail     Health Food Store   
41 -79.352188               0      Greek Restaurant           Coffee Shop   
42 -79.315572               0        Sandwich Place  Fast Food Restaurant   
43 -79.340923               0           Coffee Shop               Brewery   
44 -79.388790               3                  Park              Bus Line   

   3rd Most Common Venue   4th Most Common Venue       5th Most Common Venue  \
37           Coffee Shop                     Pub  Modern European Restaurant   
41    Italian Restaurant  Furniture / Home Store              Ice Cream Shop   
42                   Gym      Italian Restaurant                        Park   
43                Bakery               Gastropub         American Restaurant   
44           Swim School      Dim Sum Restaurant              Adult Boutique   

   6th Most Common Venue 7th Most Common Venue    8th Most Common Venue  \
37                Museum         Movie Theater      Moroccan Restaurant   
41                Lounge          Liquor Store  Fruit & Vegetable Store   
42                   Pub           Coffee Shop               Restaurant   
43                  Café           Cheese Shop         Stationery Store   
44                Museum           Men's Store            Metro Station   

   9th Most Common Venue           10th Most Common Venue  
37   Monument / Landmark  Molecular Gastronomy Restaurant  
41    Frozen Yogurt Shop                      Yoga Studio  
42         Movie Theater                          Brewery  
43             Bookstore        Middle Eastern Restaurant  
44    Mexican Restaurant        Middle Eastern Restaurant

In [69]:
address = 'Toronto, CA'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Manhattan are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Manhattan are 43.6534817, -79.3839347.


In [70]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters